In [1]:
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end

"/Users/little-orange/Documents/18Aut/238/AA228FinalProject/Project.toml"

In [2]:
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf

In [3]:
num_x_pts = 10
num_y_pts = 10
num_th_pts = 5
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts)

vlist = [i for i in 0:2:10]
omlist = [i for i in -1:0.3:1]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist))

42-element Array{RoombaAct,1}:
 [0.0, -1.0] 
 [2.0, -1.0] 
 [4.0, -1.0] 
 [6.0, -1.0] 
 [8.0, -1.0] 
 [10.0, -1.0]
 [0.0, -0.7] 
 [2.0, -0.7] 
 [4.0, -0.7] 
 [6.0, -0.7] 
 [8.0, -0.7] 
 [10.0, -0.7]
 [0.0, -0.4] 
 ⋮           
 [0.0, 0.5]  
 [2.0, 0.5]  
 [4.0, 0.5]  
 [6.0, 0.5]  
 [8.0, 0.5]  
 [10.0, 0.5] 
 [0.0, 0.8]  
 [2.0, 0.8]  
 [4.0, 0.8]  
 [6.0, 0.8]  
 [8.0, 0.8]  
 [10.0, 0.8] 

In [4]:
sensor = Bumper() # or Bumper() for the bumper version of the environment
config = 1 # 1,2, or 3
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace));

In [5]:
num_particles = 2000
# resampler = LidarResampler(num_particles, LowVarianceResampler(num_particles))
# for the bumper environment
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

In [6]:
using QMDP
solver = QMDPSolver(max_iterations=1, tolerance=1e-3)
policy = solve(solver, m)

AlphaVectorPolicy{RoombaPOMDP{Bumper,Bool},RoombaAct}(RoombaPOMDP{Bumper,Bool}(Bumper(), RoombaMDP{DiscreteRoombaStateSpace,Array{RoombaAct,1}}
  v_max: Float64 10.0
  om_max: Float64 1.0
  dt: Float64 0.5
  contact_pen: Float64 -1.0
  time_pen: Float64 -0.1
  goal_reward: Float64 10.0
  stairs_penalty: Float64 -10.0
  config: Int64 1
  room: AA228FinalProject.Room
  sspace: DiscreteRoombaStateSpace
  aspace: Array{RoombaAct}((42,))
  _amap: Dict{RoombaAct,Int64}
), Array{Float64,1}[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [8]:
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, policy, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end